<a href="https://colab.research.google.com/github/nxxk23/PROJECT/blob/main/Re_CountSlopes.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
hemo = pd.read_csv('/content/drive/MyDrive/Thesis Project Kidney/Hemodialysis.csv', encoding='latin1')
hemo.shape

(149987, 5)

In [3]:
import pandas as pd
lab = pd.read_csv('/content/drive/MyDrive/Thesis Project Kidney/Lab.csv')
lab.shape

(21641523, 11)

In [4]:
# Create a list from hemo['PatientUID']
patient_uid_list = hemo['PatientUID'].tolist()

In [5]:
# Remove patient UIDs from lab['PatientUID'] that are not in the list
lab = lab[lab['PatientUID'].isin(patient_uid_list)]
lab.shape

(3912949, 11)

In [6]:
lab.isnull().any()

LabNo              False
PatientUID         False
PatientVisitUID    False
Location            True
RequestItemName    False
ResultItemName      True
CWhen              False
ResultDttm          True
ResultValue         True
Unit                True
ReferenceRange      True
dtype: bool

##**🗂️ Data eGFR of all Patient**

In [7]:
# Define the values to match
result_item_names = ['eGFR(CKD-EPI) age >=18']

# Select rows where 'ResultItemName' matches the specified values
eGFR = lab[lab['ResultItemName'].isin(result_item_names)]
eGFR
labeGFR = eGFR[['LabNo','PatientUID', 'PatientVisitUID', 'ResultItemName', 'ResultValue', 'CWhen']]
labeGFR

,LabNo,PatientUID,PatientVisitUID,ResultItemName,ResultValue,CWhen
3592,1,2431,3571800,eGFR(CKD-EPI) age >=18,39.0,2559-10-18 11:06:38.557
3636,1,2431,4689528,eGFR(CKD-EPI) age >=18,37.80,2560-11-21 10:58:54.997
3672,1,2431,4861904,eGFR(CKD-EPI) age >=18,32.10,2561-01-24 10:38:28.390
3709,1,2431,4959141,eGFR(CKD-EPI) age >=18,31.40,2561-02-26 10:15:36.670
3765,1,2431,5107605,eGFR(CKD-EPI) age >=18,36.30,2561-04-20 10:21:19.693
...,...,...,...,...,...,...
21622569,141,2139766,7659616,eGFR(CKD-EPI) age >=18,28.15,2564-02-04 18:54:01.953
21622965,142,2139766,7659616,eGFR(CKD-EPI) age >=18,31.89,2564-02-05 01:38:17.617
21623330,143,2139766,7659616,eGFR(CKD-EPI) age >=18,36.64,2564-02-05 07:35:27.993
21623693,144,2139766,7659616,eGFR(CKD-EPI) age >=18,40.59,2564-02-05 18:48:12.583


In [8]:
labeGFR.isnull().any()

LabNo              False
PatientUID         False
PatientVisitUID    False
ResultItemName     False
ResultValue        False
CWhen              False
dtype: bool

## **🗒 Pivot each patientID**
to look at the value they have

In [9]:
import pandas as pd

# Assuming you have a DataFrame named 'eGFR' with the specified columns
pivot_table = pd.pivot_table(labeGFR, index=['PatientUID', 'PatientVisitUID'], values=['ResultItemName', 'ResultValue', 'CWhen'], aggfunc='first')
pivot_table

CWhen          ResultItemName  \
PatientUID PatientVisitUID                                                    
2431       3571800          2559-10-18 11:06:38.557  eGFR(CKD-EPI) age >=18   
           4689528          2560-11-21 10:58:54.997  eGFR(CKD-EPI) age >=18   
           4861904          2561-01-24 10:38:28.390  eGFR(CKD-EPI) age >=18   
           4959141          2561-02-26 10:15:36.670  eGFR(CKD-EPI) age >=18   
           5107605          2561-04-20 10:21:19.693  eGFR(CKD-EPI) age >=18   
...                                             ...                     ...   
3398238    10143783         2565-12-13 08:59:22.597  eGFR(CKD-EPI) age >=18   
           10165194         2565-12-19 08:37:01.873  eGFR(CKD-EPI) age >=18   
           10214038         2566-01-04 08:55:56.513  eGFR(CKD-EPI) age >=18   
           10258933         2566-01-16 08:24:59.753  eGFR(CKD-EPI) age >=18   
           10393250         2566-02-20 08:48:07.493  eGFR(CKD-EPI) age >=18   

                           ResultValue  
PatientUID PatientVisitUID              
2431       3571800                39.0  
           4689528               37.80  
           4861904               32.10  
           4959141               31.40  
           5107605               36.30  
...                                ...  
3398238    10143783              79.44  
           10165194              65.93  
           10214038              65.34  
           10258933              59.89  
           10393250              53.95  

[37299 rows x 3 columns]

In [10]:
pivot_table.isnull().any()

CWhen             False
ResultItemName    False
ResultValue       False
dtype: bool

##  **🕑 generate datetime**
    Split CWhen เพราะต้องการ generate year as คศ. Pandas มี outbound
    ให้อยู่ในระยะเวลาของ คศ. แต่ลอง convert column 'CWhen'
    เป็น datetime แล้วทำไม่ได้เลยต้อง split แล้วค่อย merge

In [11]:
import pandas as pd

df = pd.DataFrame(pivot_table)

# Splitting the 'time' column into separate columns
df[['date', 'timestamp']] = df['CWhen'].str.split(' ', expand=True)
df[['year', 'month', 'day']] = df['date'].str.split('-', expand=True)
df[['hour', 'minute', 'second']] = df['timestamp'].str.split(':', expand=True)
df[['second', 'nanosecond']] = df['second'].str.split('.', expand=True)

# Convert year column from BE to AD
df['year'] = df['year'].astype(int) - 543

# Combine columns into a datetime column
df['date'] = pd.to_datetime(df[['year', 'month', 'day',]])
# Convert timestamp to Timedelta
df['timestamp'] = pd.to_timedelta(df['timestamp'])

# Convert date to datetime
df['date'] = pd.to_datetime(df['date'])

# Merge timestamp and date columns
df['datetime'] = df['date'] + df['timestamp']

df_sorted = df.sort_values(by=['PatientUID', 'datetime'])

In [12]:
df_sorted

CWhen          ResultItemName  \
PatientUID PatientVisitUID                                                    
2431       3571800          2559-10-18 11:06:38.557  eGFR(CKD-EPI) age >=18   
           4689528          2560-11-21 10:58:54.997  eGFR(CKD-EPI) age >=18   
           4861904          2561-01-24 10:38:28.390  eGFR(CKD-EPI) age >=18   
           4959141          2561-02-26 10:15:36.670  eGFR(CKD-EPI) age >=18   
           5107605          2561-04-20 10:21:19.693  eGFR(CKD-EPI) age >=18   
...                                             ...                     ...   
3398238    10143783         2565-12-13 08:59:22.597  eGFR(CKD-EPI) age >=18   
           10165194         2565-12-19 08:37:01.873  eGFR(CKD-EPI) age >=18   
           10214038         2566-01-04 08:55:56.513  eGFR(CKD-EPI) age >=18   
           10258933         2566-01-16 08:24:59.753  eGFR(CKD-EPI) age >=18   
           10393250         2566-02-20 08:48:07.493  eGFR(CKD-EPI) age >=18   

                           ResultValue       date              timestamp  \
PatientUID PatientVisitUID                                                 
2431       3571800                39.0 2016-10-18 0 days 11:06:38.557000   
           4689528               37.80 2017-11-21 0 days 10:58:54.997000   
           4861904               32.10 2018-01-24 0 days 10:38:28.390000   
           4959141               31.40 2018-02-26 0 days 10:15:36.670000   
           5107605               36.30 2018-04-20 0 days 10:21:19.693000   
...                                ...        ...                    ...   
3398238    10143783              79.44 2022-12-13 0 days 08:59:22.597000   
           10165194              65.93 2022-12-19 0 days 08:37:01.873000   
           10214038              65.34 2023-01-04 0 days 08:55:56.513000   
           10258933              59.89 2023-01-16 0 days 08:24:59.753000   
           10393250              53.95 2023-02-20 0 days 08:48:07.493000   

                            year month day hour minute second nanosecond  \
PatientUID PatientVisitUID                                                 
2431       3571800          2016    10  18   11     06     38        557   
           4689528          2017    11  21   10     58     54        997   
           4861904          2018    01  24   10     38     28        390   
           4959141          2018    02  26   10     15     36        670   
           5107605          2018    04  20   10     21     19        693   
...                          ...   ...  ..  ...    ...    ...        ...   
3398238    10143783         2022    12  13   08     59     22        597   
           10165194         2022    12  19   08     37     01        873   
           10214038         2023    01  04   08     55     56        513   
           10258933         2023    01  16   08     24     59        753   
           10393250         2023    02  20   08     48     07        493   

                                          datetime  
PatientUID PatientVisitUID                          
2431       3571800         2016-10-18 11:06:38.557  
           4689528         2017-11-21 10:58:54.997  
           4861904         2018-01-24 10:38:28.390  
           4959141         2018-02-26 10:15:36.670  
           5107605         2018-04-20 10:21:19.693  
...                                            ...  
3398238    10143783        2022-12-13 08:59:22.597  
           10165194        2022-12-19 08:37:01.873  
           10214038        2023-01-04 08:55:56.513  
           10258933        2023-01-16 08:24:59.753  
           10393250        2023-02-20 08:48:07.493  

[37299 rows x 13 columns]

## 📈 calculate slope by **ResultValue** and **datetime**



> **Calculate slope**
```
 ให้ทำการคำนวณสโลปเฉพาะค่าที่ time_diff ห่างกันอย่างน้อย 7 วันเป็นอย่างต่ำ ถ้าน้อยกว่านั้นไม่นำมาคำนวณหาสโลป
 ให้คำนวณ time difference แบบ lag1, lag2, ..., lag6
 แล้วถ้าเจอว่า lag1 ของ time[i] (time[i] - time[i-1]) น้อยกว่าศูนย์ก็นำออก
 แต่ time[i+1] ต้องเช็คว่า lag2 < 7 ไหม ถ้าใช่ก็นำออก ถ้าไม่ใช่ก็เก็บ
```



In [13]:
import pandas as pd

# Assuming the previous code for DataFrame creation and sorting is already executed

# Function to handle conditions for lag1
def process_lag1(x):
    if x <= pd.Timedelta(0):
        return pd.NaT
    else:
        return x

# Function to handle conditions for lag2
def process_lag2(x):
    if x <= pd.Timedelta(days=7):
        return pd.NaT
    else:
        return x

# Apply the conditions for lag1 and lag2
df_sorted['lag1'] = df_sorted['lag1'].apply(process_lag1)
df_sorted['lag2'] = df_sorted['lag2'].apply(process_lag2)

# Function to determine if a row is valid or invalid based on the conditions
def is_valid(row):
    if pd.notna(row['lag1']):
        return 'NO'
    if pd.notna(row['lag2']):
        return 'NO'
    return 'YES'

# Apply the 'is_valid' function to create the 'valid' column
df_sorted['valid'] = df_sorted.apply(is_valid, axis=1)

# Drop unnecessary columns
df_sorted.drop(columns=['date', 'timestamp', 'year', 'month', 'day', 'hour', 'minute', 'second', 'nanosecond'], inplace=True)

df_sorted


KeyError: ignored

In [15]:
# Filter the DataFrame to show only rows where 'valid' column is 'YES'
valid_df = df_sorted[df_sorted['valid'] == 'YES']

# Print the DataFrame containing only the valid rows with 'YES' in the 'valid' column
valid_df

,,CWhen,ResultItemName,ResultValue,datetime,lag1,lag2,lag3,lag4,lag5,lag6,valid
PatientUID,PatientVisitUID,,,,,,,,,,,
2431,3571800,2559-10-18 11:06:38.557,eGFR(CKD-EPI) age >=18,39.0,2016-10-18 11:06:38.557,NaT,NaT,NaT,NaT,NaT,NaT,YES
2568,4593873,2560-10-19 07:24:03.470,eGFR(CKD-EPI) age >=18,68.50,2017-10-19 07:24:03.470,NaT,NaT,NaT,NaT,NaT,NaT,YES
3390,3528420,2559-10-03 08:49:31.617,eGFR(CKD-EPI) age >=18,31.9,2016-10-03 08:49:31.617,NaT,NaT,NaT,NaT,NaT,NaT,YES
5567,3352555,2559-08-02 08:44:46.440,eGFR(CKD-EPI) age >=18,12.3,2016-08-02 08:44:46.440,NaT,NaT,NaT,NaT,NaT,NaT,YES
9060,3536324,2559-10-05 09:28:22.680,eGFR(CKD-EPI) age >=18,22.6,2016-10-05 09:28:22.680,NaT,NaT,NaT,NaT,NaT,NaT,YES
...,...,...,...,...,...,...,...,...,...,...,...,...
3353511,9533342,2565-06-07 17:13:59.917,eGFR(CKD-EPI) age >=18,8.19,2022-06-07 17:13:59.917,NaT,NaT,NaT,NaT,NaT,NaT,YES
3357408,9564813,2565-06-17 13:38:54.480,eGFR(CKD-EPI) age >=18,3.67,2022-06-17 13:38:54.480,NaT,NaT,NaT,NaT,NaT,NaT,YES
3359256,9579473,2565-06-21 16:21:30.647,eGFR(CKD-EPI) age >=18,11.96,2022-06-21 16:21:30.647,NaT,NaT,NaT,NaT,NaT,NaT,YES


In [16]:
df = df_sorted[['ResultItemName', 'ResultValue','datetime']]
df

ResultItemName ResultValue  \
PatientUID PatientVisitUID                                       
2431       3571800          eGFR(CKD-EPI) age >=18        39.0   
           4689528          eGFR(CKD-EPI) age >=18       37.80   
           4861904          eGFR(CKD-EPI) age >=18       32.10   
           4959141          eGFR(CKD-EPI) age >=18       31.40   
           5107605          eGFR(CKD-EPI) age >=18       36.30   
...                                            ...         ...   
3398238    10143783         eGFR(CKD-EPI) age >=18       79.44   
           10165194         eGFR(CKD-EPI) age >=18       65.93   
           10214038         eGFR(CKD-EPI) age >=18       65.34   
           10258933         eGFR(CKD-EPI) age >=18       59.89   
           10393250         eGFR(CKD-EPI) age >=18       53.95   

                                          datetime  
PatientUID PatientVisitUID                          
2431       3571800         2016-10-18 11:06:38.557  
           4689528         2017-11-21 10:58:54.997  
           4861904         2018-01-24 10:38:28.390  
           4959141         2018-02-26 10:15:36.670  
           5107605         2018-04-20 10:21:19.693  
...                                            ...  
3398238    10143783        2022-12-13 08:59:22.597  
           10165194        2022-12-19 08:37:01.873  
           10214038        2023-01-04 08:55:56.513  
           10258933        2023-01-16 08:24:59.753  
           10393250        2023-02-20 08:48:07.493  

[37299 rows x 3 columns]

In [32]:
import pandas as pd

# Assuming you have a DataFrame named 'df_sorted' with columns 'ResultItemName', 'eGFR', 'datetime', 'PatientVisitUID', and 'time_difference' where 'PatientUID' is the index.

# Convert 'ResultValue' column to numeric
df['eGFR'] = pd.to_numeric(df_sorted['ResultValue'], errors='coerce')

# Convert 'datetime' column to datetime type
df['datetime'] = pd.to_datetime(df['datetime'])

# Drop rows with missing values in 'eGFR' and 'datetime' columns
df.dropna(subset=['eGFR', 'datetime'], inplace=True)

# Sort DataFrame by 'datetime'
df.sort_values(by=['datetime'], inplace=True)

##### Calculate the time difference in days ####
df['time_difference'] = (df['datetime'] - df['datetime'].shift()).dt.days

# Calculate slopes
df['slopes'] = (df['eGFR'].diff() / df['time_difference'])

# Select desired columns
eGFR_Patient = df[['ResultItemName', 'eGFR', 'datetime', 'slopes', 'time_difference']]


<ipython-input-32-f2695d4e78e7>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['eGFR'] = pd.to_numeric(df_sorted['ResultValue'], errors='coerce')
<ipython-input-32-f2695d4e78e7>:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['datetime'] = pd.to_datetime(df['datetime'])
<ipython-input-32-f2695d4e78e7>:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.h

In [33]:
eGFR_Patient

,,ResultItemName,eGFR,datetime,slopes,time_difference
PatientUID,PatientVisitUID,,,,,
1457398,3316712,eGFR(CKD-EPI) age >=18,20.60,2016-07-29 12:31:01.433,NaN,NaN
699218,3320725,eGFR(CKD-EPI) age >=18,33.00,2016-07-29 12:31:41.410,inf,0.0
902894,3343379,eGFR(CKD-EPI) age >=18,39.10,2016-07-29 15:19:43.850,inf,0.0
772874,3345865,eGFR(CKD-EPI) age >=18,11.10,2016-07-29 20:50:52.513,-inf,0.0
581756,3246598,eGFR(CKD-EPI) age >=18,23.30,2016-07-30 06:53:16.680,inf,0.0
...,...,...,...,...,...,...
62460,10573161,eGFR(CKD-EPI) age >=18,5.34,2023-04-10 11:12:13.893,-inf,0.0
2963972,10574487,eGFR(CKD-EPI) age >=18,43.80,2023-04-10 11:54:34.003,inf,0.0
851857,10573148,eGFR(CKD-EPI) age >=18,3.00,2023-04-10 15:44:03.403,-inf,0.0


In [19]:
t = eGFR_Patient.reset_index()
t

,PatientUID,PatientVisitUID,ResultItemName,eGFR,datetime,slopes,time_difference
0,1457398,3316712,eGFR(CKD-EPI) age >=18,20.60,2016-07-29 12:31:01.433,NaN,NaN
1,699218,3320725,eGFR(CKD-EPI) age >=18,33.00,2016-07-29 12:31:41.410,inf,0.0
2,902894,3343379,eGFR(CKD-EPI) age >=18,39.10,2016-07-29 15:19:43.850,inf,0.0
3,772874,3345865,eGFR(CKD-EPI) age >=18,11.10,2016-07-29 20:50:52.513,-inf,0.0
4,581756,3246598,eGFR(CKD-EPI) age >=18,23.30,2016-07-30 06:53:16.680,inf,0.0
...,...,...,...,...,...,...,...
37293,62460,10573161,eGFR(CKD-EPI) age >=18,5.34,2023-04-10 11:12:13.893,-inf,0.0
37294,2963972,10574487,eGFR(CKD-EPI) age >=18,43.80,2023-04-10 11:54:34.003,inf,0.0
37295,851857,10573148,eGFR(CKD-EPI) age >=18,3.00,2023-04-10 15:44:03.403,-inf,0.0
37296,1128523,10573038,eGFR(CKD-EPI) age >=18,3.09,2023-04-10 15:45:31.713,inf,0.0


# ✅ **check**


> **Check**
```
เช็คว่าแต่ละคนมีจำนวนสโลปที่คำนวณจากระยะเวลาห่างกันอย่างน้อย 3 เดือน
อย่างน้อย 2 สโลปไหม ถ้ามีไม่ครบให้ตัดคนนั้นทิ้ง
```


In [20]:
import pandas as pd

# Assuming you have a DataFrame 'eGFR_Patient' with index 'PatientUID' and columns 'datetime', 'slopes', and 'time_difference'

# Create an empty list to store the valid PatientUIDs
valid_patient_uids = []

# Group the DataFrame by index 'PatientUID'
grouped = eGFR_Patient.groupby(level='PatientUID')

# Iterate over each group
for patient_uid, group in grouped:
    # Check if the group has at least 2 rows
    if len(group) >= 2:
        # Calculate the minimum and maximum datetime in the group
        min_datetime = group['datetime'].min()
        max_datetime = group['datetime'].max()

        # Calculate the time difference in days between the minimum and maximum datetime
        time_diff = (max_datetime - min_datetime).days

        # Check if the time difference is at least 90 days (3 months)
        if time_diff >= 90:
            valid_patient_uids.append(patient_uid)

# Filter the original DataFrame based on the valid PatientUIDs
filtered_df = eGFR_Patient.loc[valid_patient_uids]

In [21]:
filtered_df

ResultItemName   eGFR  \
PatientUID PatientVisitUID                                  
2431       3571800          eGFR(CKD-EPI) age >=18  39.00   
           4689528          eGFR(CKD-EPI) age >=18  37.80   
           4861904          eGFR(CKD-EPI) age >=18  32.10   
           4959141          eGFR(CKD-EPI) age >=18  31.40   
           5107605          eGFR(CKD-EPI) age >=18  36.30   
...                                            ...    ...   
3398238    10143783         eGFR(CKD-EPI) age >=18  79.44   
           10165194         eGFR(CKD-EPI) age >=18  65.93   
           10214038         eGFR(CKD-EPI) age >=18  65.34   
           10258933         eGFR(CKD-EPI) age >=18  59.89   
           10393250         eGFR(CKD-EPI) age >=18  53.95   

                                          datetime  slopes  time_difference  
PatientUID PatientVisitUID                                                   
2431       3571800         2016-10-18 11:06:38.557    -inf              0.0  
           4689528         2017-11-21 10:58:54.997     inf              0.0  
           4861904         2018-01-24 10:38:28.390     inf              0.0  
           4959141         2018-02-26 10:15:36.670     inf              0.0  
           5107605         2018-04-20 10:21:19.693    -inf              0.0  
...                                            ...     ...              ...  
3398238    10143783        2022-12-13 08:59:22.597     inf              0.0  
           10165194        2022-12-19 08:37:01.873     inf              0.0  
           10214038        2023-01-04 08:55:56.513     inf              0.0  
           10258933        2023-01-16 08:24:59.753     inf              0.0  
           10393250        2023-02-20 08:48:07.493     inf              0.0  

[37295 rows x 5 columns]

In [22]:
row_counts = filtered_df.groupby('PatientUID').size()


patient_uids_with_less_than_2_row = row_counts[row_counts < 2].index

patient_uids_with_2_row = row_counts[row_counts == 2].index

print("PatientUIDs with less than 2 row:") #dont have
print(patient_uids_with_less_than_2_row)

print("PatientUIDs with 2 row:")
print(patient_uids_with_2_row)

PatientUIDs with less than 2 row:
Int64Index([], dtype='int64', name='PatientUID')
PatientUIDs with 2 row:
Int64Index([  95886,  255435,  270357,  304855,  314905,  320289,  370784,
             440026,  507307,  544072,  643342,  646905,  720399,  760223,
             786431,  804478,  812332,  833364,  884564,  886409,  967060,
            1042341, 1049193, 1071859, 1077503, 1078291, 1094701, 1111129,
            1185293, 1192158, 1233203, 1251081, 1272179, 1273059, 1337646,
            1343071, 1368970, 1380776, 1420637, 1434440, 1457823, 1475635,
            1494903, 1508811, 1532109, 1533924, 1542653, 1546930, 1553540,
            1583948, 1590267, 1622575, 1652679, 1693013, 1840839, 1889348,
            1953986, 2039355, 2068378, 2181364, 2206752, 2352850, 2372836,
            2376745, 2454320, 2467300, 2551254, 2556448, 2564781, 2579457,
            2623015, 2677599, 2762253, 2782638, 2809643, 2834211, 2991646,
            3016240, 3034062, 3043980, 3070639, 3091627, 3125154, 31

In [23]:
test = filtered_df.reset_index() # check if time_diff > 90
test.head()

,PatientUID,PatientVisitUID,ResultItemName,eGFR,datetime,slopes,time_difference
0,2431,3571800,eGFR(CKD-EPI) age >=18,39.0,2016-10-18 11:06:38.557,-inf,0.0
1,2431,4689528,eGFR(CKD-EPI) age >=18,37.8,2017-11-21 10:58:54.997,inf,0.0
2,2431,4861904,eGFR(CKD-EPI) age >=18,32.1,2018-01-24 10:38:28.390,inf,0.0
3,2431,4959141,eGFR(CKD-EPI) age >=18,31.4,2018-02-26 10:15:36.670,inf,0.0
4,2431,5107605,eGFR(CKD-EPI) age >=18,36.3,2018-04-20 10:21:19.693,-inf,0.0


In [24]:
t[t['PatientUID']==3417416] # before cut

,PatientUID,PatientVisitUID,ResultItemName,eGFR,datetime,slopes,time_difference


In [25]:
test[test['PatientUID']==3417416] # check if time_diff > 90

,PatientUID,PatientVisitUID,ResultItemName,eGFR,datetime,slopes,time_difference


- **Delete each first rows slope of each PatientUID**

In [26]:
# Replace 'slopes' and 'time_difference' columns of the first row for each unique 'PatientUID' in DataFrame 'filtered_df'
for id in filtered_df.index.get_level_values('PatientUID').unique():
    first_row_index = filtered_df.loc[(id, slice(None)), :].index[0]
    filtered_df.loc[first_row_index, ('slopes', 'time_difference')] = None

# Print the updated DataFrame 'filtered_df'
filtered_df

ResultItemName   eGFR  \
PatientUID PatientVisitUID                                  
2431       3571800          eGFR(CKD-EPI) age >=18  39.00   
           4689528          eGFR(CKD-EPI) age >=18  37.80   
           4861904          eGFR(CKD-EPI) age >=18  32.10   
           4959141          eGFR(CKD-EPI) age >=18  31.40   
           5107605          eGFR(CKD-EPI) age >=18  36.30   
...                                            ...    ...   
3398238    10143783         eGFR(CKD-EPI) age >=18  79.44   
           10165194         eGFR(CKD-EPI) age >=18  65.93   
           10214038         eGFR(CKD-EPI) age >=18  65.34   
           10258933         eGFR(CKD-EPI) age >=18  59.89   
           10393250         eGFR(CKD-EPI) age >=18  53.95   

                                          datetime  slopes  time_difference  
PatientUID PatientVisitUID                                                   
2431       3571800         2016-10-18 11:06:38.557     NaN              NaN  
           4689528         2017-11-21 10:58:54.997     inf              0.0  
           4861904         2018-01-24 10:38:28.390     inf              0.0  
           4959141         2018-02-26 10:15:36.670     inf              0.0  
           5107605         2018-04-20 10:21:19.693    -inf              0.0  
...                                            ...     ...              ...  
3398238    10143783        2022-12-13 08:59:22.597     inf              0.0  
           10165194        2022-12-19 08:37:01.873     inf              0.0  
           10214038        2023-01-04 08:55:56.513     inf              0.0  
           10258933        2023-01-16 08:24:59.753     inf              0.0  
           10393250        2023-02-20 08:48:07.493     inf              0.0  

[37295 rows x 5 columns]

In [27]:
a = filtered_df.reset_index()

In [28]:
a

,PatientUID,PatientVisitUID,ResultItemName,eGFR,datetime,slopes,time_difference
0,2431,3571800,eGFR(CKD-EPI) age >=18,39.00,2016-10-18 11:06:38.557,NaN,NaN
1,2431,4689528,eGFR(CKD-EPI) age >=18,37.80,2017-11-21 10:58:54.997,inf,0.0
2,2431,4861904,eGFR(CKD-EPI) age >=18,32.10,2018-01-24 10:38:28.390,inf,0.0
3,2431,4959141,eGFR(CKD-EPI) age >=18,31.40,2018-02-26 10:15:36.670,inf,0.0
4,2431,5107605,eGFR(CKD-EPI) age >=18,36.30,2018-04-20 10:21:19.693,-inf,0.0
...,...,...,...,...,...,...,...
37290,3398238,10143783,eGFR(CKD-EPI) age >=18,79.44,2022-12-13 08:59:22.597,inf,0.0
37291,3398238,10165194,eGFR(CKD-EPI) age >=18,65.93,2022-12-19 08:37:01.873,inf,0.0
37292,3398238,10214038,eGFR(CKD-EPI) age >=18,65.34,2023-01-04 08:55:56.513,inf,0.0
37293,3398238,10258933,eGFR(CKD-EPI) age >=18,59.89,2023-01-16 08:24:59.753,inf,0.0


In [29]:
a['PatientUID'].nunique()

1607

In [30]:
a.to_excel('SlopeFiltered.xlsx', index=False)


**suggestion**
```
ควรจะเก็บแค่สโลปที่มีมากกว่า 3 rows เพราะถ้าเก็บสโลปอย่างน้อย 2 rows (2 slopes)
some patient will have only 1 slope cuz the fist slope must be deleted

```

